In [4]:
#Importing libraries

import cv2
import numpy as np
from myYolo import myYolo
import trackObject as tob
from random import randint


#Initialising yolo model with myYolo class from myYolo.py file
model=myYolo()

#Capturing video from vide file. You can change to 0 for webcam
cap=cv2.VideoCapture('demo1.mp4')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)

result = cv2.VideoWriter('demo1_dete.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)


frame_count=0

#Initialising multiTracker
multiTracker = cv2.MultiTracker_create()
trackerType = "CSRT" 

p_boxes=[]
colors=[]
d_arr=[]
person_count=0

#Starting Infinte loop
while True:
    frame_count+=1
    ret,img=cap.read()
    
    #If no frame is detected, break out of loop
    if not ret:
        break
          
    #Initialising objects to track
    if frame_count==1:
        
        #Updating Yolo model
        model.make_model(img)
        
        #Find objects from frame. Class findObjects in myYolo.py
        model.findObjects()
        
        p_boxes,colours=model.find_boxes()
        for box in p_boxes:
            #Initising boxes in multitraker
            multiTracker.add(tob.createTrackerByName(trackerType), img, (box[0],box[1],box[2],box[3]))
            person_count+=1
            
    #Updating detection once in 5 frames
    if frame_count%5==0:
        boxes=[]
        model.make_model(img)
        model.findObjects()
        boxes,colours=model.find_boxes()
        p_boxes,indxs,new_add,d_=tob.update_boxes(p_boxes,boxes,d_th=100)
        person_count+=new_add
        
        #Updating new detections in multiTraker
        for box in p_boxes:
            multiTracker.add(tob.createTrackerByName(trackerType), img, (box[0],box[1],box[2],box[3]))
    
    #Updating traking boxes
    ret, p_boxes = multiTracker.update(img)
    
    #Checking social distance violation with check_distance function in tob.py
    red_boxes=tob.check_distance(p_boxes,250)
    if not ret:
        break
   
    #Drawing box and puting text in ouput    
    for i, newbox in enumerate(p_boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        tob.drawBox(img,p1,p2,(0,255,0)) 
        cv2.putText(img,f"P{i}",p1,cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,255),1)
    for newbox in red_boxes:
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        tob.drawBox(img,p1,p2,(0,0,255))
       
    cv2.putText(img,f"{frame_count}, personcount: {person_count}",(200,75),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,0,255),1)
    cv2.imshow('MultiTracker', img)
    result.write(img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()
result.release()
    
    